In [1]:
from pyspark.sql import SparkSession

number_cores = 6
memory_gb = 10
name_app = 'pib'

spark = (SparkSession.builder.appName(name_app) 
    .config('spark.driver.memory', f'{memory_gb}g')
    .config('spark.master', f'local[{number_cores}]')
    ).getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/11 09:28:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from os import walk

path_root = '/media/davi/6A81-05CF/DATALAKES/emg_dataset/' 
folder = 'parquet'

f = []
for (dirpath, dirnames, filenames) in walk(path_root + folder):
    f.extend(dirnames)
    break

In [17]:
def load_data(file:str) -> any:
    df = (spark.read.format('parquet').load(f"{path_root}{folder}/{file}"))
    df.cache()

    return df

def sep_cols(df:any) -> list:
    eeg_cols = list(filter(lambda col: 'EEG' in col, df.columns ))
    ecg_cols = list(filter(lambda col: 'ECG' in col, df.columns ))
    emg_cols = list(filter(lambda col: 'EMG' in col, df.columns ))

    return [eeg_cols, ecg_cols, emg_cols]


def main(file:str) -> None:

    df = load_data(file)

    cols = sep_cols(df)

    sub_folders = ['eeg','ecg','emg']

    for col,folder in zip(cols, sub_folders):
        try:
            (df.select('time','window',*col)
                .write.format('parquet')
                .partitionBy('window')
                .mode('overwrite')
                .save(f"{path_root}{folder}/{file}/" ))
        except Exception as e:
            print(e)


In [19]:
for file in f:
    main(file)

23/02/11 09:54:59 WARN CacheManager: Asked to cache already cached data.        


In [20]:
spark.stop()